In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [3]:
train.dtypes

EmployeeNo                              object
Division                                object
Qualification                           object
Gender                                  object
Channel_of_Recruitment                  object
Trainings_Attended                       int64
Year_of_birth                            int64
Last_performance_score                 float64
Year_of_recruitment                      int64
Targets_met                              int64
Previous_Award                           int64
Training_score_average                   int64
State_Of_Origin                         object
Foreign_schooled                        object
Marital_Status                          object
Past_Disciplinary_Action                object
Previous_IntraDepartmental_Movement     object
No_of_previous_employers                object
Promoted_or_Not                          int64
dtype: object

In [4]:
train.duplicated().sum()

0

In [5]:
#Since there are no duplicate rows, drop EmployeeNo column since it is uninformative
train.drop(['EmployeeNo'], axis=1, inplace=True)

In [6]:
#It is better to work with number of years rather than a given year value. 
train['Age'] = 2019 - train['Year_of_birth']
train['Years_on_job'] = 2019 - train['Year_of_recruitment']
train.drop(['Year_of_birth', 'Year_of_recruitment'], axis=1, inplace=True)

In [7]:
train.describe()

,Trainings_Attended,Last_performance_score,Targets_met,Previous_Award,Training_score_average,Promoted_or_Not,Age,Years_on_job
count,38312.000000,38312.000000,38312.000000,38312.000000,38312.000000,38312.000000,38312.000000,38312.000000
mean,2.253680,7.698959,0.352996,0.023152,55.366465,0.084595,32.790666,5.860305
std,0.609443,3.744135,0.477908,0.150388,13.362741,0.278282,7.646047,4.261451
min,2.000000,0.000000,0.000000,0.000000,31.000000,0.000000,18.000000,1.000000
25%,2.000000,5.000000,0.000000,0.000000,43.000000,0.000000,27.000000,3.000000
50%,2.000000,7.500000,0.000000,0.000000,52.000000,0.000000,31.000000,5.000000
75%,2.000000,10.000000,1.000000,0.000000,68.000000,0.000000,37.000000,7.000000
max,11.000000,12.500000,1.000000,1.000000,91.000000,1.000000,69.000000,37.000000


It looks like both Targets_met and Previous_Award only take values [0, 1]. It is possible to confirm this. Promoted_or_Not is our target.

In [8]:
for col in ['Targets_met', 'Previous_Award']:
    print("{}: {}".format(col, train[col].nunique()))

Targets_met: 2
Previous_Award: 2


Both columns are better treated as categorical. 

In [9]:
train[['Previous_Award', 'Targets_met']] = train[['Previous_Award', 'Targets_met']].astype('object')

In [10]:
train['Promoted_or_Not'].value_counts()

0    35071
1     3241
Name: Promoted_or_Not, dtype: int64

The train data is imabalanced. This is important to note for later steps.

# 1.0 Handling Missing Data

In [11]:
train.isnull().sum()

Division                                  0
Qualification                          1679
Gender                                    0
Channel_of_Recruitment                    0
Trainings_Attended                        0
Last_performance_score                    0
Targets_met                               0
Previous_Award                            0
Training_score_average                    0
State_Of_Origin                           0
Foreign_schooled                          0
Marital_Status                            0
Past_Disciplinary_Action                  0
Previous_IntraDepartmental_Movement       0
No_of_previous_employers                  0
Promoted_or_Not                           0
Age                                       0
Years_on_job                              0
dtype: int64

In [12]:
test.isnull().sum()

EmployeeNo                               0
Division                                 0
Qualification                          730
Gender                                   0
Channel_of_Recruitment                   0
Trainings_Attended                       0
Year_of_birth                            0
Last_performance_score                   0
Year_of_recruitment                      0
Targets_met                              0
Previous_Award                           0
Training_score_average                   0
State_Of_Origin                          0
Foreign_schooled                         0
Marital_Status                           0
Past_Disciplinary_Action                 0
Previous_IntraDepartmental_Movement      0
No_of_previous_employers                 0
dtype: int64

Both the train and test are missing some Qualification data. Dropping rows with missing values is not an option here. The options are:

1. Fill with the most frequent category. 
2. Predict missing values. 
3. Use an algorithm that supports missing values.
4. Coding missing values as another category.

Option 1 & 2 can be compared to help choice of strategy.
The missing data can be extracted as a dataframe and explored for insights.

In [13]:
#Create seoarate dataframes for train data with and without missing qualification values
missing_data = train.loc[train['Qualification'].isnull()]
no_missing_data = train.drop(index=missing_data.index).reset_index(drop=True)

#Create a copy to be used in subsequent manipulations
train_no_missing_data = no_missing_data.copy()

missing_data.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Last_performance_score,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not,Age,Years_on_job
15,Customer Support and Field Operations,NaN,Male,Direct Internal process,2,10.0,0,0,49,RIVERS,Yes,Married,No,No,1,0,39,11
22,Commercial Sales and Marketing,NaN,Female,Direct Internal process,2,2.5,0,0,40,EDO,Yes,Married,No,No,1,0,22,2
28,Commercial Sales and Marketing,NaN,Male,Agency and others,4,5.0,0,0,40,CROSS RIVER,Yes,Married,No,No,1,0,22,2
60,Commercial Sales and Marketing,NaN,Female,Direct Internal process,2,0.0,0,0,47,ZAMFARA,Yes,Single,No,No,2,0,18,1
137,Customer Support and Field Operations,NaN,Female,Agency and others,2,10.0,0,0,56,LAGOS,Yes,Single,No,No,5,0,31,9


## Strategy 1 - Filling With Missing Values

In [14]:
def test_mode_strategy(data, column, pct_test=0.3, num_runs=5, data_contains_missing_values=False):
    
    """Takes a dataframe and checks the average accuracy of filling missing values with the mode.
    
    # data - the DataFrame
    # column - the column of the Dataframe where missing values are present
    # pct_test - the % of data to be used as test set. default is 30%
    # num_runs - the number of trials over which to average accuracy of the strategy
    # data_contains_missing_values - default is False. This indicates whether data passed
    # to function still contains missing values.""" 
    
    df = data.copy()
    df = df[str(column)]
    
    #Drops missing values
    if data_contains_missing_values:
        df.dropna(inplace=True)
    
    #Picks the most frequent value
    mode = df.value_counts().idxmax()
    
    count = 0
    acc = 0
    for i in range(num_runs):
        
        #Create randomly selected train and test dataframes
        df_test = df.sample(frac=pct_test)
        
        count += 1
        acc += df_test.loc[df_test == mode].shape[0] / df_test.shape[0]
        
    accuracy = acc / count
    
    print("Filling with the most frequent category yields a {:.1f}% accuracy".format(accuracy*100))      

In [15]:
test_mode_strategy(train_no_missing_data, column='Qualification', num_runs=10)

Filling with the most frequent category yields a 69.9% accuracy


## Strategy 2 - Predicting Missing Values

In [16]:
categorical_columns = list(train_no_missing_data.select_dtypes(include=['object']).columns)

numerical_columns = list(train_no_missing_data.select_dtypes(exclude=['object']).columns)

In [17]:
train_no_missing_data.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Last_performance_score,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not,Age,Years_on_job
0,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,12.5,1,0,41,ANAMBRA,No,Married,No,No,0,0,33,8
1,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,12.5,0,0,52,ANAMBRA,Yes,Married,No,No,0,0,28,4
2,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,7.5,0,0,42,KATSINA,Yes,Married,No,No,0,0,32,7
3,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,2.5,0,0,42,NIGER,Yes,Single,No,No,1,0,37,10
4,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,7.5,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0,29,7


In [18]:
numerical_columns

['Trainings_Attended',
 'Last_performance_score',
 'Training_score_average',
 'Promoted_or_Not',
 'Age',
 'Years_on_job']

In [19]:
target = train_no_missing_data['Qualification']
train_no_missing_data.drop(['Qualification'], axis=1, inplace=True)
#Necessary to onehot encoide the target as Neural Networks require it
target = pd.get_dummies(target)
target.head()

,First Degree or HND,"MSc, MBA and PhD",Non-University Education
0,0,1,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [20]:
#Remove the target column 
categorical_columns.remove('Qualification')

#Retrieve the index of the categorical and numerical columns 
cat_index = [list(train_no_missing_data.columns).index(x) for x in categorical_columns]
num_index = [list(train_no_missing_data.columns).index(x) for x in numerical_columns]

In [21]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer 

ct = ColumnTransformer([("onehot", OneHotEncoder(drop='first'), categorical_columns),
                        ("scaler", MinMaxScaler(), numerical_columns)],
                      remainder='passthrough',
                      sparse_threshold=0)

ct.fit(train_no_missing_data)
train_no_missing_data_onehot = ct.transform(train_no_missing_data)

train_no_missing_data_onehot[:1]

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.16666667, 0.        , 0.29411765,
        0.19444444]])

In [22]:
train_no_missing_data_onehot.shape

(36633, 66)

In [23]:
from sklearn.model_selection import train_test_split
part_x_train, part_x_test, part_y_train, part_y_test = train_test_split(train_no_missing_data_onehot,
                                                                        target, stratify=target, 
                                                                        test_size=0.3)

In [24]:
import tensorflow
from keras import models, layers

network = models.Sequential()
network.add(layers.Dense(16, activation='relu', input_shape=(66,)))
network.add(layers.Dense(32, activation='relu'))
network.add(layers.Dense(3, activation='softmax'))
network.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

part_x_test = train_no_missing_data_onehot[:5000]
part_y_test = target[:5000]
part_x_train = train_no_missing_data_onehot[5000:]
part_y_train = target[5000:]

count = 0
acc = 0
for i in range(5):
    history = network.fit(part_x_train,
                        part_y_train,
                        epochs=25,
                        verbose=0,
                        batch_size=512)
    acc += network.evaluate(part_x_test, part_y_test)[1]
    count += 1
    
accuracy = acc / count
print("Predicting with a Neural Network yields a {:.1f}% accuracy".format(accuracy*100))

c:\users\oladipo a.f\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\oladipo a.f\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\oladipo a.f\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c


5000/5000 [==============================] - 0s 54us/step
Predicting with a Neural Network yields a 71.9% accuracy


Over 5 runs, predicting Qualification data with Neural Network achieves a higher average accuracy than filling with the most frequent. It makes sense to try this strategy as well as using models that support missing values.

The other two strategies - using a model that supports missing values and labelling missing values with a unique category - will be compared later on when the model is being trained and tuned. 

# Using the Neural Network To Predict the Actual Missing Data in Training Set

In [25]:
missing_data.drop(['Qualification'], axis=1, inplace=True)

c:\users\oladipo a.f\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
missing_data_transformed = ct.transform(missing_data)
missing_data_transformed[:1]

array([[0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.8       , 0.3       , 0.        , 0.41176471,
        0.27777778]])

In [27]:
network.fit(train_no_missing_data_onehot,
            target,
            epochs=25,
            verbose=0, 
            batch_size=512)

pred_qual = network.predict(missing_data_transformed)
pred_qual = np.argmax(pred_qual, axis=1)

In [28]:
pred_qual[:5]

array([0, 0, 0, 0, 0], dtype=int64)

In [29]:
labels = {}
for col in target.columns:
    labels[list(target.columns).index(col)] =  col

In [30]:
labels

{0: 'First Degree or HND',
 1: 'MSc, MBA and PhD',
 2: 'Non-University Education'}

In [31]:
pred_qual = [labels[qual] for qual in pred_qual]

In [32]:
missing_data['Qualification'] = pred_qual

c:\users\oladipo a.f\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
missing_data.head()

,Division,Gender,Channel_of_Recruitment,Trainings_Attended,Last_performance_score,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not,Age,Years_on_job,Qualification
15,Customer Support and Field Operations,Male,Direct Internal process,2,10.0,0,0,49,RIVERS,Yes,Married,No,No,1,0,39,11,First Degree or HND
22,Commercial Sales and Marketing,Female,Direct Internal process,2,2.5,0,0,40,EDO,Yes,Married,No,No,1,0,22,2,First Degree or HND
28,Commercial Sales and Marketing,Male,Agency and others,4,5.0,0,0,40,CROSS RIVER,Yes,Married,No,No,1,0,22,2,First Degree or HND
60,Commercial Sales and Marketing,Female,Direct Internal process,2,0.0,0,0,47,ZAMFARA,Yes,Single,No,No,2,0,18,1,First Degree or HND
137,Customer Support and Field Operations,Female,Agency and others,2,10.0,0,0,56,LAGOS,Yes,Single,No,No,5,0,31,9,First Degree or HND


In [34]:
train_new = pd.concat([no_missing_data, missing_data], ignore_index=True, sort=True)
train_new.head()

,Age,Channel_of_Recruitment,Division,Foreign_schooled,Gender,Last_performance_score,Marital_Status,No_of_previous_employers,Past_Disciplinary_Action,Previous_Award,Previous_IntraDepartmental_Movement,Promoted_or_Not,Qualification,State_Of_Origin,Targets_met,Training_score_average,Trainings_Attended,Years_on_job
0,33,Direct Internal process,Commercial Sales and Marketing,No,Female,12.5,Married,0,No,0,No,0,"MSc, MBA and PhD",ANAMBRA,1,41,2,8
1,28,Agency and others,Customer Support and Field Operations,Yes,Male,12.5,Married,0,No,0,No,0,First Degree or HND,ANAMBRA,0,52,2,4
2,32,Direct Internal process,Commercial Sales and Marketing,Yes,Male,7.5,Married,0,No,0,No,0,First Degree or HND,KATSINA,0,42,2,7
3,37,Agency and others,Commercial Sales and Marketing,Yes,Male,2.5,Single,1,No,0,No,0,First Degree or HND,NIGER,0,42,3,10
4,29,Direct Internal process,Information and Strategy,Yes,Male,7.5,Married,1,No,0,No,0,First Degree or HND,AKWA IBOM,0,77,3,7


In [35]:
train_new.isnull().sum()

Age                                    0
Channel_of_Recruitment                 0
Division                               0
Foreign_schooled                       0
Gender                                 0
Last_performance_score                 0
Marital_Status                         0
No_of_previous_employers               0
Past_Disciplinary_Action               0
Previous_Award                         0
Previous_IntraDepartmental_Movement    0
Promoted_or_Not                        0
Qualification                          0
State_Of_Origin                        0
Targets_met                            0
Training_score_average                 0
Trainings_Attended                     0
Years_on_job                           0
dtype: int64

All missing qualification have been predicted now. Time for further exploration and some visualisation!

In [36]:
train_new.to_csv('new_train.csv', index=False)

The cleaned dataset can be stored to csv for easy retrieval. 